In [3]:
from torchvision import transforms
import onnxruntime
import numpy as np 
from PIL import Image

In [7]:
sz = 299
model = onnxruntime.InferenceSession("model/angle_mobileNet_20.onnx")
transforms =  transforms.Compose([
        transforms.Resize((sz , sz)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [13]:
img = Image.open("1.jpg").convert('RGB')
img = transforms(img)
img = np.array(img)

img = np.expand_dims(img, 0).astype(np.float32)
    
ort_outs = model.run(None, {"input":img})

ort_outs[0][0][0]

0.082428336

## test class

In [2]:
import glob
from natsort import natsorted
from Stabilizer import Stabilizer

stab = Stabilizer("model/yolov8s_6_2023.pt" , "model/angle_inception_16.onnx")
imgs = glob.glob("octo_org/*")

c = 1 
for im in natsorted(imgs):
    stab.run(im ,"out(inception-center-zoom)/"+str( c)+".jpg")
    c+=1
    print(c)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


## trad stab

In [ ]:
import cv2 , os
from natsort import natsorted
import numpy as np
import matplotlib.pyplot as plt
from stabilize import *

name = "result"

imgs = []

directory = "OUT1/octo-sampling-model_out"

# Read all frame filenames from the directory
sorted_items = os.listdir(directory)
sorted_items = natsorted(sorted_items)
for filename in sorted_items:
    if filename.endswith(".png") or filename.endswith(".jpg"):
        frame_path = os.path.join(directory, filename)
        frame = cv2.imread(frame_path)
        
        if frame is not None:
            imgs.append(frame)



ws = create_warp_stack(imgs)



#calculate the smoothed trajectory and output the zeroed images
smoothed_warp, smoothed_trajectory, original_trajectory = moving_average(ws, sigma_mat= np.array([[5,5, 5],[5 , 5 , 5]]))
new_imgs = apply_warping_fullview(images=imgs, warp_stack=ws-smoothed_warp, PATH='./out/')




    # Define the video codec, output format, and frame rate
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video = cv2.VideoWriter("octo_combined_sampling10xx.mp4", fourcc, 8, (new_imgs[0].shape[1], new_imgs[0].shape[0]))

# Write frames to the video
for frame in new_imgs:
   video.write(frame)

# Release the video writer and destroy any remaining windows
video.release()
cv2.destroyAllWindows()

#plot the original and smoothed trajectory
f, (a0, a1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[1, 1]})

i,j = 0,2
a0.scatter(np.arange(len(original_trajectory)), np.array(original_trajectory)[:,i,j],s=2 , label='Original')
a0.plot(np.arange(len(original_trajectory)), np.array(original_trajectory)[:,i,j])
a0.scatter(np.arange(len(smoothed_trajectory)), np.array(smoothed_trajectory)[:,i,j], s=2 , label='Smoothed')
a0.plot(np.arange(len(smoothed_trajectory)), np.array(smoothed_trajectory)[:,i,j])
a0.legend()
a0.set_ylabel('X trajectory')
a0.xaxis.set_ticklabels([])

i,j = 0,1
a1.scatter(np.arange(len(original_trajectory)), np.array(original_trajectory)[:,i,j],s=2, label='Original')
a1.plot(np.arange(len(original_trajectory)), np.array(original_trajectory)[:,i,j])
a1.scatter(np.arange(len(smoothed_trajectory)), np.array(smoothed_trajectory)[:,i,j],s=2, label='Smoothed')
a1.plot(np.arange(len(smoothed_trajectory)), np.array(smoothed_trajectory)[:,i,j])
a1.legend()
a1.set_xlabel('Frame')
a1.set_ylabel('Sin(Theta) trajectory')
plt.savefig(name+'_smoothed.png')

#create a images that show both the trajectory and video frames
filenames = imshow_with_trajectory(images=new_imgs, warp_stack=ws-smoothed_warp, PATH='ou'+'/', ij=(0,2))